In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('mimic-acute-mi.csv')
df.head()

,hadm_id,ecg_time,st_elevation,st_depression,t_wave,Acute_MI,study_id,text,valuenum,comments,troponin,STEMI,NSTEMI
0,29196424.0,2087-05-08 00:05:00,0,0,0,0,44673611,Allergies: \nlisinopril / vancomycin\n \nChief...,NaN,NaN,0,0,0
1,21091437.0,2087-05-08 00:05:00,0,0,0,0,43819938,Allergies: \namitriptyline / Sulfa (Sulfonamid...,0.03,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
2,28951019.0,2087-05-08 00:05:00,0,0,0,0,44864920,Allergies: \nPenicillins\n \nChief Complaint:\...,NaN,<0.01. cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
3,21236438.0,2088-04-14 02:58:00,0,0,0,0,47239325,Allergies: \nMULTIPLE - SEE LIST **** / amoxic...,0.02,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0
4,23821411.0,2089-12-11 00:06:00,0,0,0,0,41010651,Allergies: \nchlorhexidine\n \nChief Complaint...,0.07,cTropnT > 0.10 ng/mL suggests Acute MI.,1,0,0


In [3]:
with open('conv_templates.json') as f:
    conv_templates = json.load(f)

In [4]:
df = df[~df['text'].isna()]
df = df[(df['troponin'] & ~df['valuenum'].isnull() & ~df['comments'].isnull()) | (df['troponin'] == 0)]

In [5]:
conversations = []

for idx, row in df.iterrows():
    conv = {
        "id": idx,
        "image": f"{idx}.jpeg",
        "conversations": []
    }

    if row['STEMI']:
        conv['diagnosis'] = 'STEMI'
    elif row['NSTEMI']:
        conv['diagnosis'] = 'NSTEMI'
    else:
        conv['diagnosis'] = 'NO ACS'

    conv['conversations'].append({
        "from": "human",
        "value" : conv_templates['st_elev_q'][0].replace('{INSERT PATIENT NOTE}', row['text'])
    })

    if row['st_elevation']:
        conv['conversations'].append({
            "from": "gpt",
            "value" : conv_templates['st_elev_a_true'][0]
        })
        conv['conversations'].append({
            "from": "human",
            "value" : conv_templates['troponin_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
        })
        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['troponin_a_true'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['diag_q_trop'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value']).replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['troponin_a_false'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['diag_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })

        if row['STEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['diag_a_stemi'][0]
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['diag_a_normal'][0]
            })
    else:
        conv['conversations'].append({
            "from": "gpt",
            "value" : conv_templates['st_elev_a_false'][0]
        })
        conv['conversations'].append({
            "from": "human",
            "value" : conv_templates['st_depres_twi_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
        })
        if row['st_depression'] or row['t_wave']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['st_depres_twi_a_true'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['troponin_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['st_depres_twi_a_false'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['troponin_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })

        if row['troponin']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['troponin_a_true'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['diag_q_trop'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value']).replace('{INSERT_RESULT}', str(row['valuenum']) + ',' + row['comments'])
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['troponin_a_false'][0]
            })
            conv['conversations'].append({
                "from": "human",
                "value" : conv_templates['diag_q'][0].replace('{INSERT PRIOR ANSWER}', conv['conversations'][-1]['value'])
            })
            
        if row['NSTEMI']:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['diag_a_nstemi'][0]
            })
        else:
            conv['conversations'].append({
                "from": "gpt",
                "value" : conv_templates['diag_a_normal'][0]
            })
    conversations.append(conv)


In [6]:
with open('conversations.json', 'w') as f:
    json.dump(conversations, f)
